# Función de Perdida

Todos los algoritmos en Deep Learning dependen de minimizar alguna función, la *función objetivo*, que es conocida como la *función de perdida*. Una función de perdida nos permite medir que tan bien está prediciendo el resultado esperado un modelo predictivo.


## Mean Square Error

Utilizado en problemas de regresión. Conforme el valor del error aumente, mayor será la penalización.

\begin{align}
MSE = \frac{1}{n}\sum^n_{i=1}(y_i-\hat{y_i})^2
\end{align}

## Cross-Entropy

Utilizado en problemas de clasificación. Al igual que en MSE, la penalización aumenta conforme el error. Sin embargo, la penalización en esta función es más fuerte que la de MSE.

\begin{align}
ACE = -\frac{1}{n}\sum^n_{i=1}log(\frac{e^{f_{y_i}}}{\sum_j e^{f_j}})
\end{align}

Donde $y_i$ es el indicador de la clase que debería ser correcta, $f$ es el valor de cada clase obtenido por la red, y $j$ el indicador de los valores de las clases predichas.

Notese quela operación interna del $log$ es la operación de softmax.

\begin{align}
softmax = \frac{e^{f_{y_i}}}{\sum_j e^{f_j}}
\end{align}

In [1]:
import numpy as np

In [2]:
# Mean Square Error
def loss_mse(y,y_pred):
    SE = (y-y_pred)**2
    MSE = np.mean(SE)
    return(MSE)

def loss_mse_elem(y,y_pred):
    SE = (y-y_pred)**2
    MSE = np.mean(SE,axis=-1)
    return(MSE)

In [3]:
# Average Cross-Entropy
def loss_ace(y,y_pred):
    log = np.log(np.sum(y*y_pred,axis=-1))
    ace = -1*np.mean(log)
    return(ace)

def loss_ace_elem(y,y_pred):
    log = np.log(np.sum(y*y_pred,axis=-1))
    ace = -1*log
    return(ace)

## Ejemplo: Regresión

Tenemos una red que quiere aproximar sus resultados a una función $f(x)$. Sus resultados están mostrados en *y_pred*. Debido a que queremos minimizar la distancia entre los valores predecidos y los valores reales, utilizamos *Mean Square Error* para determinar el error entre estos. Notese que mientras más alejados estén los valores de su valor real, MSE penaliza en mayor medida el error.

In [4]:
# Valores predecidos
y_pred = np.array([[0.8],[2.4],[5.1],[8]])
# Valores reales
y = np.array([[1],[3],[5],[7]])

Calculamos el error utilizando MSE.

$SE_0 = (1-0.8)^2 = 0.04$

$MSE_{error} = (0.04 + 0.36 + 0.01 + 1)/4 = 0.35$

In [5]:
mse = loss_mse(y,y_pred)

print(mse)

0.35250000000000004


In [6]:
mse = loss_mse_elem(y,y_pred)

print(mse)

[0.04 0.36 0.01 1.  ]


#### ¿Es posible utilizar Cross-Entropy para este caso? ¿Porqupe?

## Ejemplo: MSE vs Cross-Entropy

Imaginemos que tenemos dos redes neuronales que predicen la probabilidad que tiene una persona a pertenecer a un partido partido politico-PRI, PAN, MORENA. Esta es una tarea de clasificación.


Como base de datos tenemos la preferencia de tres personas.

| PRI | PAN | MORENA|
|-----|
| 0 | 0 | 1 |
| 0 | 1 | 0 |
| 1 | 0 | 1 |

La primera red tiene una predicción como la siguiente.

| PRI | PAN | MORENA |
|-----|
| 0.3 | 0.3 | 0.4 |
| 0.3 | 0.4 | 0.3 |
| 0.2 | 0.2 | 0.7 |

La segunda red tiene la siguiente predicción.

| PRI | PAN | MORENA |
|-----|
| 0.1 | 0.2 | 0.7 |
| 0.1 | 0.7 | 0.2 |
| 0.3 | 0.4 | 0.3 |

In [7]:
# Predicciones red 1
y_pred1 = np.array([[0.3,0.3,0.4],
                   [0.3,0.4,0.3],
                   [0.1,0.2,0.7]])

# Predicciones red 2
y_pred2 = np.array([[0.1,0.2,0.7],
                    [0.1,0.7,0.2],
                    [0.3,0.4,0.3]])

# Valores reales
y = np.array([[0,0,1],
              [0,1,0],
              [1,0,0]])

Un par de observaciones.

#### Error de clasificación
Ambas neuronas han predecido las primeras dos pertenencias correctamente, y la ultima incorrectamente, y las dos tienen un mismo error de clasificación a pesar de que la segunda tuvo una predicción más acercada a la realidad.

Error de la red #1: 1/3

Error de la red #2: 1/3

#### Error Cuadrático Promedio
Si calculamos el MSE de ambos resultados, este nos da valores apropiados. La red #1 presenta un error cuadrpatico promedio más grande que la red #2. Justo como debería de ser.

Para la primera red y el primer elemento de los resultados, se calcula de la siguiente manera:
$(0-0.3)^2 +(0-0.3)^2 + (1-0.4)^2 = 0.18$

Que para calcular el MSE completo se procede como sigue:
$(0.18 + 0.18 + 0.44)/3 = 0.27 $

MSE de la red #1: 0.27

MSE de la red #2: 0.11

#### Error Cross-Entropy
Sin embargo, al observar los resultados que obtenemos al utilizar cross-entropy, y a sabiendas del logaritmo utilizado, nos podemos dar cuenta de que el utilizar cross-entropy nos permite penalizar más fuertemente a las predicciones conforme a que tan alejadas estén de la clase real. Además, esta función nos ayuda a tener un mejor entrenamiento ya que los gradientes se ven disminuidos en menor proporción con respecto a MSE conforme se acercan las predicciones a su valor deseado. Por otro lado, notese la importancia que le da MSE a los valores que no corresponden a la clase correcta que fue incorrectamente predecida

Para la primera red y el primer elemento de los resultados, se calcula de la siguiente manera:
$-log(0.3)*0 -log(0.3)*0 -log(0.4)*1 = 0.92$

Que para calcular el ACE completo se procede como sigue:
$(0.92 + 0.92 + 2.3)/3 = 1.38$

ACE de la red #1: 1.38

ACE de la red #2: 0.64

In [8]:
# Mean Square Error Completo para la red 1 y 2
mse1 = loss_mse(y,y_pred1)
mse2 = loss_mse(y,y_pred2)

# Average Cross-Entropy para la red 1 y 2
ace1 = loss_ace(y,y_pred1)
ace2 = loss_ace(y,y_pred2)

print(mse1,mse2)
print(ace1,ace2)

0.2688888888888889 0.11333333333333334
1.3783888522474517 0.6391075640678003


In [9]:
# Square Error para cada elemento de los resultados en la red 1 y 2
mse1 = loss_mse_elem(y,y_pred1)
mse2 = loss_mse_elem(y,y_pred2)

# Cross-Entropy para cada elemento de los resultados en la red 1 y 2
ace1 = loss_ace_elem(y,y_pred1)
ace2 = loss_ace_elem(y,y_pred2)

print(mse1,mse2)
print(ace1,ace2)

[0.18       0.18       0.44666667] [0.04666667 0.04666667 0.24666667]
[0.91629073 0.91629073 2.30258509] [0.35667494 0.35667494 1.2039728 ]
